In [1]:
!pip install kafka-python

In [2]:
!pip install pyspark

In [3]:
!python3.11 -m pip install --upgrade pip

In [4]:
import json
from time import sleep
from json import dumps
import pandas as pd
import requests
from kafka import KafkaProducer
from pyspark.sql import SparkSession
from pyspark.sql.functions import get_json_object

In [5]:
# Create a Spark session
spark = SparkSession.builder \
    .appName("test_kafka_aws") \
    .getOrCreate()

24/08/31 23:43:49 WARN Utils: Your hostname, Harshs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.183.81.8 instead (on interface en0)
24/08/31 23:43:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/31 23:43:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
BASE_URL = "https://financialmodelingprep.com/api/v3/stock_market/actives"
API_KEY = "API_KEY"

In [7]:
producer = KafkaProducer(bootstrap_servers=[':9092'],#Add IP/dns
                         value_serializer=lambda x: 
                         dumps(x).encode('utf-8'))

In [8]:
URL = BASE_URL + API_KEY
print(URL)

https://financialmodelingprep.com/api/v3/stock_market/actives?apikey=0npNunB1WcSdfxa7Nzs0DWrIVnp0AHNn


In [9]:
response = requests.get(URL)
df = pd.DataFrame(response.json())
df

,symbol,name,change,price,changesPercentage
0,WTO,UTime Limited,-0.0150,0.0756,-16.5563
1,NVDA,NVIDIA Corporation,1.7800,119.3700,1.5137
2,FCUV,Focus Universal Inc.,0.0787,0.2689,41.3775
3,BNRG,Brenmiller Energy Ltd,0.8770,1.5500,130.3120
4,INTC,Intel Corporation,1.9100,22.0400,9.4883
5,SQQQ,ProShares UltraPro Short QQQ,-0.2700,8.2000,-3.1877
6,NU,Nu Holdings Ltd.,0.7400,14.9700,5.2003
7,MAXN,"Maxeon Solar Technologies, Ltd.",0.0038,0.1081,3.6433
8,SOXL,Direxion Daily Semiconductor Bull 3X Shares,2.6000,38.7900,7.1843
9,ATPC,Agape ATP Corporation Common Stock,0.9580,2.5500,60.1759


In [ ]:
#calls limitation if 24 hours usage -> 250/day => 1 every 345.6 seconds. Once approx 6 minutes (360 seconds).

In [21]:
producer.send("test_kafka_aws", response.json())
producer.flush()  # Ensure all messages are sent

In [23]:
for i in range(10):
    #get data from API
    response = requests.get(URL)
    producer.send("test_kafka_aws", response.json())
    producer.flush()
    sleep(5)

KeyboardInterrupt: 

In [10]:
producer.send('test_kafka_aws', value={"Hi": "Hello"}) #test

In [24]:
producer.close()